In [1]:
import heapq
from collections import deque

graph_1 = {  
    'a': [('b', 9), ('c', 4), ('d', 7)],  
    'b': [('e', 11)],  
    'c': [('e', 17), ('f', 12)],  
    'd': [('f', 14)],  
    'e': [('z', 5)],  
    'f': [('z', 9)],  
    'z': []  
}  

heuristics_1 = {  
    'a': 21, 'b': 14, 'c': 18, 'd': 18, 
    'e': 5, 'f': 8, 'z': 0  
}  

start_1 = 'a'
goal_1 = 'z'
BEAM_WIDTH_1 = 2

def reconstruct_path(parent, start, goal):
    path = []
    current = goal
    while current != start:
        if current not in parent or parent[current] is None:
            return None
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()
    return path

def best_first_search(graph, heuristics, start, goal):
    open_set = []
    heapq.heappush(open_set, (heuristics[start], start))
    parent = {start: None}
    
    while open_set:
        h, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(parent, start, goal)
        for neighbor, _ in graph.get(current, []):
            if neighbor not in parent:
                heapq.heappush(open_set, (heuristics[neighbor], neighbor))
                parent[neighbor] = current
    return None

def beam_search(graph, heuristics, start, goal, beam_width):
    queue = deque([[start]])
    while queue:
        candidates = []
        current_beam_size = len(queue)
        for _ in range(current_beam_size):
            path = queue.popleft()
            node = path[-1]
            if node == goal:
                return path
            for neighbor, _ in graph.get(node, []):
                new_path = path + [neighbor]
                candidates.append((heuristics[neighbor], new_path))
        if not candidates:
            break
        candidates.sort(key=lambda x: x[0])
        queue = deque([path for h, path in candidates[:beam_width]])
    return None

# ---------------------- RUN ----------------------
print(f"--- Task 4.a: Graph 1 ({start_1} to {goal_1}) ---")
path_bfs_1 = best_first_search(graph_1, heuristics_1, start_1, goal_1)
print(f"1. Best First Search Path: {' -> '.join(path_bfs_1) if path_bfs_1 else 'Path not found'}")

path_beam_1 = beam_search(graph_1, heuristics_1, start_1, goal_1, BEAM_WIDTH_1)
print(f"2. Beam Search (k={BEAM_WIDTH_1}) Path: {' -> '.join(path_beam_1) if path_beam_1 else 'Path not found'}")


--- Task 4.a: Graph 1 (a to z) ---
1. Best First Search Path: a -> b -> e -> z
2. Beam Search (k=2) Path: a -> b -> e -> z


In [2]:
import heapq
from collections import deque

# ---------------- GRAPH 2 ----------------
graph_2 = {  
    'S': [('A', 4), ('B', 10), ('C', 11)],  
    'A': [('B', 8), ('D', 5)],  
    'B': [('D', 15)],  
    'C': [('D', 8), ('E', 20), ('F', 2)],  
    'D': [('H', 16), ('I', 20), ('F', 1)],  
    'E': [('G', 19)],  
    'F': [('G', 13)],  
    'H': [('I', 1), ('J', 2)],  
    'I': [('G', 5), ('K', 13)],  
    'J': [('K', 7)],  
    'K': [('G', 16)],  
    'G': []  
}

heuristics_2 = {  
    'S': 7, 'A': 8, 'B': 6, 'C': 5, 'D': 5,  
    'E': 3, 'F': 3, 'G': 0, 'H': 7, 'I': 4,  
    'J': 5, 'K': 3  
}

start_2 = 'S'
goal_2 = 'G'
BEAM_WIDTH_2 = 2

# ---------------- Helper ----------------
def reconstruct_path(parent, start, goal):
    path = []
    current = goal
    while current != start:
        if current not in parent or parent[current] is None:
            return None
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()
    return path

# ---------------- Best-First Search ----------------
def best_first_search(graph, heuristics, start, goal):
    open_set = []
    heapq.heappush(open_set, (heuristics[start], start))
    parent = {start: None}
    
    while open_set:
        h, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(parent, start, goal)
        for neighbor, _ in graph.get(current, []):
            if neighbor not in parent:
                heapq.heappush(open_set, (heuristics[neighbor], neighbor))
                parent[neighbor] = current
    return None

# ---------------- Beam Search ----------------
def beam_search(graph, heuristics, start, goal, beam_width):
    queue = deque([[start]])
    while queue:
        candidates = []
        current_beam_size = len(queue)
        for _ in range(current_beam_size):
            path = queue.popleft()
            node = path[-1]
            if node == goal:
                return path
            for neighbor, _ in graph.get(node, []):
                new_path = path + [neighbor]
                candidates.append((heuristics[neighbor], new_path))
        if not candidates:
            break
        candidates.sort(key=lambda x: x[0])
        queue = deque([path for h, path in candidates[:beam_width]])
    return None

# ---------------- A* Search ----------------
def a_star_search(graph, heuristics, start, goal):
    open_set = []
    heapq.heappush(open_set, (heuristics[start], 0, start, [start]))
    best_g_cost = {start: 0}
    
    while open_set:
        f, g, current, path = heapq.heappop(open_set)
        if current == goal:
            return path, g
        if g > best_g_cost.get(current, float('inf')):
            continue
        for neighbor, cost in graph.get(current, []):
            new_g = g + cost
            if new_g < best_g_cost.get(neighbor, float('inf')):
                best_g_cost[neighbor] = new_g
                new_h = heuristics.get(neighbor, float('inf'))
                new_f = new_g + new_h
                new_path = path + [neighbor]
                heapq.heappush(open_set, (new_f, new_g, neighbor, new_path))
    return None, float('inf')

# ---------------- Run Searches ----------------
print(f"--- Task 4.b: Graph 2 ({start_2} to {goal_2}) ---")

# Best-First Search
path_bfs_2 = best_first_search(graph_2, heuristics_2, start_2, goal_2)
cost_bfs = sum(cost for i, node in enumerate(path_bfs_2) if i > 0 
               for neighbor, cost in graph_2.get(path_bfs_2[i-1], []) if neighbor == node) \
           if path_bfs_2 else 'N/A'
print(f"1. Best First Search Path: {' -> '.join(path_bfs_2) if path_bfs_2 else 'Path not found'}")
print(f"   Total Cost (Non-Optimal): {cost_bfs}")

# Beam Search
path_beam_2 = beam_search(graph_2, heuristics_2, start_2, goal_2, BEAM_WIDTH_2)
cost_beam = sum(cost for i, node in enumerate(path_beam_2) if i > 0 
                for neighbor, cost in graph_2.get(path_beam_2[i-1], []) if neighbor == node) \
            if path_beam_2 else 'N/A'
print(f"2. Beam Search (k={BEAM_WIDTH_2}) Path: {' -> '.join(path_beam_2) if path_beam_2 else 'Path not found'}")
print(f"   Total Cost (Non-Optimal): {cost_beam}")

# A* Search
path_astar_2, cost_astar_2 = a_star_search(graph_2, heuristics_2, start_2, goal_2)
print(f"3. A* Search Path: {' -> '.join(path_astar_2) if path_astar_2 else 'Path not found'}")
print(f"   A* Total Cost (Optimal): {cost_astar_2 if path_astar_2 else 'N/A'}")


--- Task 4.b: Graph 2 (S to G) ---
1. Best First Search Path: S -> C -> E -> G
   Total Cost (Non-Optimal): 50
2. Beam Search (k=2) Path: S -> C -> E -> G
   Total Cost (Non-Optimal): 50
3. A* Search Path: S -> A -> D -> F -> G
   A* Total Cost (Optimal): 23
